In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from copy import deepcopy


In [90]:
def present(idn):
    d={}
    for i,idn in enumerate(idn):
        d[idn]=i
    return d  
#/my_data/
train=pd.read_csv('/my_data/train.csv',index_col=0)
validate=pd.read_csv('/my_data/validate.csv',index_col=0)

# train=train[:50000]
# validate=validate[:20000]

users=sorted(list(set(train.index).union(set(validate.index))))
nb_users=len(users)
business=sorted(list(set(train['business_id']).union(set(validate['business_id']))))
nb_business=len(business)
print(nb_users,nb_business)
dic=present(business)
# Converting the data into an array with users in lines and movies in columns

def convert(data,batch_size):
    #new_data = []
    #data=data.iloc[0:(nb_users//batch_size)*batch_size,:]
    for i,id_users in enumerate(users):
        
        id_business = data['business_id'][data.index == id_users]
        ind=list(map(lambda x:dic[x],id_business))
        #id_business = list(map(lambda x:business.index(x),l))
        id_ratings = data['stars'][data.index == id_users]
        ratings = np.zeros(nb_business)
        ratings[ind] = id_ratings
        if i%batch_size==0:
            if i!=0:
                yield np.array(U)
            U=[list(ratings)]
        else:
            U.append(list(ratings))
        #new_data.append(list(ratings))
    #return new_data
        
    
train_set = convert(train,500)
val_set = convert(validate,500)

C:\Users\fzhan\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


259122 54345


In [91]:
class LFM:
    
    def __init__(self,train_set,nb_users,nb_business,factor,batch_size,iter_num = 20,alpha = 0.1,Lambda = 0.1,epsilon = 0.01):
        '''
        initiate parameters
        '''
        self.train_set = train_set
        self.nb_users=nb_users
        self.nb_business=nb_business
        self.factor = factor
        self.batch_size=batch_size
        self.batchs=nb_users//batch_size
        self.alpha = alpha
        self.iter_num = iter_num
        self.Lambda = Lambda
        self.epsilon = epsilon
        
    
    def build(self,train):
        

        #initiate latent factor matrix
        self.decompose_p = tf.Variable(tf.ones([self.nb_business,self.factor]))
        self.decompose_q = tf.Variable(tf.ones([self.factor,self.nb_users]))    
        
        # Placeholders

        self.u_ind = tf.placeholder('int32', shape=[self.batch_size])
        #i = tf.placeholder('int32')
        self.y_rate = tf.placeholder('float32', shape=[self.batch_size, self.nb_business])
        self.y_mask = tf.placeholder('float32', shape=[self.batch_size, self.nb_business])
        #batch_q = tf.placeholder('float32', shape=[self.batch_size, self.factor])

        batch_q=tf.gather(tf.transpose(self.decompose_q),self.u_ind)
        self.pred_y_rate = tf.matmul(batch_q, tf.transpose(self.decompose_p))
        loss_m = tf.squared_difference(self.y_rate, self.pred_y_rate)
        self.loss = tf.reduce_sum(loss_m * self.y_mask)+self.Lambda*(tf.reduce_sum(tf.square(self.decompose_p))+tf.reduce_sum(tf.square(batch_q)))
        optimizer = tf.train.AdamOptimizer(self.alpha)
        train_op = optimizer.minimize(self.loss)
                                                      

        #training
        config = tf.ConfigProto() 
        config.gpu_options.per_process_gpu_memory_fraction = 0.4
        self.sess = tf.Session(config=config)
        self.sess.run(tf.initialize_all_variables())
        
    
        old_error = float('inf')
        
        tot_loss = 0.0
        for epoch in range(self.iter_num):
            train_set = convert(train,500)
            t = time.time()
            
            for i,U in enumerate(train_set):
                #with self.sess.as_default():
                #    batch_q=tf.transpose(self.decompose_q).eval()[i*self.batch_size:(i+1)*self.batch_size,:]  
                u_ind=list(range(i*self.batch_size,(i+1)*self.batch_size))
                mask=deepcopy(U)
                mask[mask!=0]=1
                feed={self.y_rate:U,self.y_mask:mask,self.u_ind:u_ind}
                _, loss = self.sess.run([train_op, self.loss], feed_dict = feed)
                tot_loss += loss
                #print (i)
                avg_loss = tot_loss / (epoch*self.batchs+i+1)
                if i%10==0:
                    print ("Epoch %d\tLoss\t%.2f\tTime %dmin" \
                    % (epoch, avg_loss, (time.time()-t)))
            


        print ("Recommender is built!")
        
    def test(self,val_set):
        """
        :return performance on test set (Mean Square Root Error)
        """
        tot_loss = 0.0
        for i,U in enumerate(val_set):
            u_ind=list(range(i*self.batch_size,(i+1)*self.batch_size))
            mask=deepcopy(U)
            mask[mask!=0]=1
            feed={self.y_rate:U,self.y_mask:mask,self.u_ind:u_ind}
            loss = self.sess.run(self.loss, feed_dict = self.get_feed_dict(self.reader.get_next_test()))
            tot_loss += loss
        return tot_loss / self.batchs
        
        

In [92]:
import time
sess=tf.InteractiveSession()
tf.global_variables_initializer().run()
lfm=LFM(train_set,nb_users,nb_business,factor=10,batch_size=500,iter_num = 10,alpha = 0.15)

lfm.build(train)
print (lfm.decompose_p)
print (lfm.decompose_q)

loss=lfm.test(val_set)
print(loss)
# ex = range(len(lfm.delta_error))
# plt.figure(1)
# plt.plot(ex,lfm.delta_error)
# plt.show()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 0	Loss	107242.00	Time 10min


KeyboardInterrupt: 